In [2]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import Dataset,DataLoader
import time
import torchvision.datasets.mnist as ds
from PIL import Image
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline



In [3]:
def has_file_allowed_extension(filename, extensions):
    """Checks if a file is an allowed extension.
    Args:
        filename (string): path to a file
        extensions (iterable of strings): extensions to consider (lowercase)
    Returns:
        bool: True if the filename ends with one of given extensions
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)

def make_dataset(dir, class_to_idx, extensions):
    images = []
    dir = os.path.expanduser(dir)
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if has_file_allowed_extension(fname, extensions):
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    images.append(item)

    return images

class TinyMindDataset(Dataset):
    """TInymind contest dataset"""
    def __init__(self, root_dir, extensions, transform=None):
        classes, class_to_idx = self._find_classes(root_dir)
        self.files = make_dataset(root_dir, class_to_idx, extensions)
        self.transform = transform
        self.extensioons = extensions
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        path, target = self.files[idx]
        img = Image.open(path).resize((64,64))
        
        if self.transform is not None:
            img = self.transform(img)
            
        return img, target
    
    def _find_classes(self, dir):
        classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx
    

In [4]:
transform = transforms.Compose([transforms.ToTensor()])
tiny_dataset = TinyMindDataset(root_dir='/home/hannahzhang/tinymind/train/', extensions=['jpg'], transform=transform)
data_size = len(tiny_dataset)
split = (int)(data_size / 10)
indices = list(range(data_size))
np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)



train_dataloader = DataLoader(tiny_dataset, batch_size=10, shuffle=False, sampler=train_sampler, num_workers=2)
valid_dataloader = DataLoader(tiny_dataset, batch_size=10, shuffle=False, sampler=valid_sampler, num_workers=2)

In [5]:
# from torchvision import models
# vgg = models.vgg16(pretrained=True)

In [6]:
# for param in vgg.features.parameters(): param.requires_grad = False
# vgg.features[0] = nn.Conv2d(1, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
# vgg.classifier[6].out_features = 100
# vgg_optimizer = optim.SGD(vgg.classifier.parameters(), lr=0.01, momentum=.65)


In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 32, kernel_size = 3)
        self.conv3 = nn.Conv2d(32, 32, kernel_size = 3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1152, 100)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)
    
    

In [ ]:
USE_GPU = True

print_every = 100

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    

def check_accuracy(loader, model): 
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
            
def train(model, optimizer, loader_train, loader_val, epochs=1):
    model = model.to(device=device)
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            loss = F.cross_entropy(scores, y)
            
            optimizer.zero_grad()
            
            loss.backward()
            
            optimizer.step()
            
            if t % print_every == 0:
                print('Iteration {}, loss= {}'.format(t, loss.item()))
                check_accuracy(loader_val, model)
                print()
    

model = Net()

optimizer = optim.SGD(model.parameters(), lr=1e-1)

train(model, optimizer, train_dataloader, valid_dataloader, epochs=200)

Iteration 0, loss= 4.623625755310059
Got 31 / 4000 correct (0.78)

Iteration 100, loss= 4.576729774475098
Got 43 / 4000 correct (1.07)

Iteration 200, loss= 4.603630542755127
Got 54 / 4000 correct (1.35)

Iteration 300, loss= 4.580783367156982


In [ ]:
print("done")

In [ ]:
# train(vgg, vgg_optimizer, train_dataloader, valid_dataloader, epochs=20)

In [ ]:
print("done")

In [ ]:
torch.save(model.state_dict(), "/home/hannahzhang/learn_pytorch/tiny")
